## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-17-21-53-07 +0000,bbc,Starmer says Trump's tariff threat over Greenl...,https://www.bbc.com/news/articles/c78ene3221yo...
1,2026-01-17-21-50-59 +0000,nypost,Laugh your a** off at ‘The Naked Comedy Show’ ...,https://nypost.com/2026/01/17/us-news/naked-co...
2,2026-01-17-21-48-19 +0000,bbc,Trump tariff threat over Greenland 'unacceptab...,https://www.bbc.com/news/articles/cy4qjwk9n2no...
3,2026-01-17-21-47-26 +0000,nyt,"At Least $110,000 Worth of Pokémon Cards Stole...",https://www.nytimes.com/2026/01/17/nyregion/po...
4,2026-01-17-21-40-49 +0000,nypost,"Ukrainian officials in Miami for peace talks, ...",https://nypost.com/2026/01/17/world-news/ukrai...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2400/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,44
23,new,12
105,iran,12
215,leader,11
175,ice,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
87,2026-01-17-13-33-32 +0000,nypost,Iran’s supreme leader blasts Trump as a ‘crimi...,https://nypost.com/2026/01/17/world-news/ayato...,87
148,2026-01-17-04-00-00 +0000,wsj,"Entering the second year of his second term, P...",https://www.wsj.com/politics/policy/over-two-w...,84
64,2026-01-17-17-03-12 +0000,nypost,"Trump slaps increased tariffs on Denmark, Euro...",https://nypost.com/2026/01/17/world-news/how-t...,83
153,2026-01-17-03-06-57 +0000,cbc,Trump invites Carney to join 'Board of Peace' ...,https://www.cbc.ca/news/world/carney-invite-tr...,81
7,2026-01-17-21-34-17 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...,81


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
87,87,2026-01-17-13-33-32 +0000,nypost,Iran’s supreme leader blasts Trump as a ‘crimi...,https://nypost.com/2026/01/17/world-news/ayato...
175,60,2026-01-17-00-59-04 +0000,cbc,U.S. Justice Department probing Minnesota Gov....,https://www.cbc.ca/news/world/tim-walz-jacob-f...
69,45,2026-01-17-16-49-16 +0000,nypost,Ex-Nickelodeon child star Kianna Underwood’s d...,https://nypost.com/2026/01/17/us-news/ex-nicke...
58,41,2026-01-17-17-50-20 +0000,bbc,Blair and Rubio among names on senior executiv...,https://www.bbc.com/news/articles/c07xv92vrz2o...
148,40,2026-01-17-04-00-00 +0000,wsj,"Entering the second year of his second term, P...",https://www.wsj.com/politics/policy/over-two-w...
7,37,2026-01-17-21-34-17 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...
74,35,2026-01-17-15-58-37 +0000,nypost,Family of college student killed in Iranian pr...,https://nypost.com/2026/01/17/world-news/famil...
9,34,2026-01-17-21-29-53 +0000,nypost,California’s largest hospital group owes patie...,https://nypost.com/2026/01/17/us-news/kaiser-p...
154,28,2026-01-17-03-05-41 +0000,nyt,Judge Restricts Immigration Agents’ Actions To...,https://www.nytimes.com/2026/01/16/us/minnesot...
55,27,2026-01-17-17-55-58 +0000,cbc,"European Union, South America's Mercosur bloc ...",https://www.cbc.ca/news/world/trade-agreement-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
